# 전국 신규 민간 아파트 분양가격 동향
- 2013년부터 최근까지 부동산 가격 변동 추세를 알아봅니다.
- 전혀 다른 형태의 두 데이터를 가져오 ㅏ정제하고 병합하는 과정을 다루는 방법 학습합니다
- 전처리한 결과에 대해 수치형, 범주형 데이터를 이해하고 다양한 그래프로 시각화합니다

## Config

In [ ]:
import pandas as pd

---

## Preprocessing

### Default

#### load

In [ ]:
df_first = pd.read_csv('./condominium/전국 평균 분양가격(2013년 9월부터 2015년 8월까지).csv', encoding='cp949')
df_last = pd.read_csv('./condominium/주택도시보증공사_전국 평균 분양가격(2019년 12월).csv', encoding='cp949')

####  check

In [ ]:
df_first.shape

In [ ]:
df_last.shape

In [ ]:
df_first.info()

In [ ]:
df_last.info()

#### Missing Data

In [ ]:
df_last.isnull().sum()

##### 데이터타입 변경
- 분양가격이 object(문자)타입이라 계산이 불가합니다.
- 계산을 수행하기위해서는 수치데이터로 변경하여야 하는데, 결측치가 포함되어있으면 변환이 의도한대로 되지않을 수 있습니다.
- to_numeric을 사용할 경우, 데이터타입 변경시 별도의 예외처리가 가능합니다.
    - __errors__ : [
        'coerce' : 사전적 의미는 '강압'이고, type이 float인 NaN값으로 변환
    ]


In [ ]:
df_last['분양가격'] = pd.to_numeric(df_last['분양가격(㎡)'], errors='coerce')
df_last['분양가격'].head()

In [ ]:
df_last.info()

######  넘파이의 NaN값은 float

In [ ]:
type(pd.np.nan)

### 평당 가격 구하기
- 현재 사용중인 2013년부터의 데이터는 평당분양가격 기준으로 되어 있습니다.
- 분양가격을 평당기준으로 보기위해 3.3을 곱해 '평당분양가격'컬럼을 추가합니다.

In [ ]:
df_last['평당분양가격'] = df_last['분양가격'] * 3.3 # 평당분양가격 컬럼 추가 (평당분양가격 = 분양가격 * 3.3)
df_last.head()

In [ ]:
df_last.info() # 평당분양가격 데이터 확인

### 규모구분을 전용면적 컬럼으로 변경
- 규모구분 컬럼은 전용면적에 대한 내용이 있습니다.
- 전용면적이라는 문구가 공통적으로 들어가고 규모구분보다는 전용면적이 좀 더 직관적이기 때문에 전용면적이라는 컬럼을 새로 만들어주고 기존 규모구분의 값에서 전용면적, 초과, 이하 등의 문구를 빼고 __간결하게 저장__합니다.

- str의 replace기능을 사용해 "전용면적 60㎡초과 85㎡이하"라면 "60㎡~85㎡"로 변경

In [ ]:
df_last['전용면적'] = df_last['규모구분'].str.replace('전용면적', "") # "전용면적" -> ""
df_last['전용면적'] = df_last['전용면적'].str.replace('초과', "~") # "초과" -> "~"
df_last['전용면적'] = df_last['전용면적'].str.replace('이하', "") # "이하" -> ""
df_last['전용면적'] = df_last['전용면적'].str.replace('" "', "") # " " -> ""
df_last['전용면적']

### 불필요 컬럼 삭제

In [ ]:
df_last = df_last.drop(['규모구분', '분양가격(㎡)'], axis=1)
df_last.head()

### 데이터 집계 with groupby

#### 지역별 평당분양 평균가

In [ ]:
df_last.groupby(by=['지역명'])['평당분양가격'].mean()

#### 전용면적별 평당분양 평균가

In [ ]:
df_last.groupby(by=['전용면적'])['평당분양가격'].mean()

#### 전용면적, 지역명별 평당분양 평균가

In [ ]:
df_last.groupby(by=['전용면적', '지역명'])['평당분양가격'].mean()

In [ ]:
df_last.groupby(by=['전용면적', '지역명'])['평당분양가격'].mean().unstack().round()

#### 연도, 지역별 평당분양가격

In [ ]:
g = df_last.groupby(by=['연도', '지역명'])['평당분양가격'].mean().unstack().transpose().round()
g

### 데이터 집계 with pivot_table
- pivot_table()함수는 구성속성중 aggfunc의 default가 mean
    - 대상 values 컬럼이 숫자면 mean
    - 대상 values 컬럼이 문자면 count

#### 지역별 평당분양 평균가

In [ ]:
pd.pivot_table(df_last, index=['지역명'], values=['평당분양가격'], aggfunc='mean').round()

#### 전용면적별 평당분양 평균가

In [ ]:
pd.pivot_table(df_last, index=['전용면적'], values=['평당분양가격'], aggfunc='mean').round()

#### 전용면적, 지역명별 평당분양 평균가

In [ ]:
pd.pivot_table(df_last, index=['전용면적', '지역명'], values=['평당분양가격'], aggfunc='mean').round()

#### 연도, 지역별 평당분양가격

In [ ]:
pd.pivot_table(df_last, index=['연도', '지역명'], values=['평당분양가격'], aggfunc='mean').round()

In [ ]:
pd.pivot_table(df_last, index=['연도', '지역명'], values=['평당분양가격'], aggfunc='mean').unstack().transpose().round()

## Visualization

### Plot

---

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('default')
plt.rc("font", family="Malgun Gothic")
plt.rc('axes', unicode_minus=False)

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

---

#### Line Chart

In [ ]:
g = df_last.groupby(['지역명'])['평당분양가격'].mean().sort_values(ascending=False)
g.plot() # 기본값이 line

#### Bar Chart

In [ ]:
g.plot.bar(figsize=(10, 4))

---
> #### Box-and-Whisker plot | Diagram
>> 가공하지 않은 자료 그대로를 이용하여 그린 것이 아니라, 자료로부터 얻어낸 5가지 통계요약수치를 그립니다.
>>5가지 요약 수치란 기술통계학에서 자료의 정보를 알려주는 아래의 다섯가지 수치를 의미
   >> 1. 최대값
   >> 1. 제 3사분위수
   >> 1. 제 2사분위수(=중앙값)
   >> 1. 제 1사분위수
   >> 1. 최소값
   --- 

In [ ]:
df_last.pivot_table(index='월', columns='연도', values='평당분양가격')

In [ ]:
df_last.pivot_table(index='월', columns='연도', values='평당분양가격').plot.box()

In [ ]:
p = df_last.pivot_table(index='월', columns=['연도', '전용면적'], values='평당분양가격')
p.plot.box(figsize=(18, 5), rot=30) # rot : x라벨 기울임 정도

### Seaborn

---

In [ ]:
import seaborn as sns

---

#### Line Chart

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data=df_last, x='연도', y='평당분양가격', hue='지역명', ci=None)
plt.legend(bbox_to_anchor=(1.02, 1), loc=2)

#### Bar Chart

In [ ]:
sns.barplot(data=df_last, x='연도', y='평당분양가격', ci=None)

#### Cat Chart

In [ ]:
sns.catplot(data=df_last, x='연도', y='평당분양가격', kind='bar', col='지역명', col_wrap=4, ci=None)

#### Rel Chart

In [ ]:
sns.relplot(data=df_last, x='연도', y='평당분양가격', hue='지역명', kind='line', col='지역명', col_wrap=3, ci=None)

#### Cat Chart
- searborn에서 제공되는 boxplot에서 matplot에서 제공되는 boxplot 값들과 이상치에대한 값들이 추가로 표시됩니다.
- 최소값보다 사분위수 범위 값(3사분위값-1사분위값)에 해당하는 값을 초과하는 값

In [ ]:
sns.boxplot(data=df_last, x='연도', y='평당분양가격')

#### Violin Chart
- boxplot에 데이터에 대한 분포량까지 표시하는 내용이 포함된 차트이고, 바이올린처럼 생겨서 이름이 바이올린 플랏입니다.

In [ ]:
sns.violinplot(data=df_last, x='연도', y='평당분양가격')

# 2015년 8월 이전 데이터 보기

In [300]:
pd.options.display.max_columns = 25

## Preprocessing

### Default

####  check

In [301]:
df_first.head(3)

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,2014년9월,2014년10월,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,19404,19759,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443
1,부산,8111,8111,9078,8965,9402,9501,9453,9457,9411,9258,9110,9208,9208,9204,9235,9279,9327,9345,9515,9559,9581
2,대구,8080,8080,8077,8101,8267,8274,8360,8360,8370,8449,8403,8439,8253,8327,8416,8441,8446,8568,8542,8542,8795


In [313]:
df_last.head() # df_first와 형식이 다름. first데이터를 last형식에 맞추는 작업 진행

,지역명,연도,월,분양가격,평당분양가격,전용면적
0,서울,2015,10,5841.0,19275.3,전체
1,서울,2015,10,5652.0,18651.6,60㎡
2,서울,2015,10,5882.0,19410.6,60㎡~ 85㎡
3,서울,2015,10,5721.0,18879.3,85㎡~ 102㎡
4,서울,2015,10,5879.0,19400.7,102㎡~


In [314]:
df_first.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   지역        17 non-null     object
 1   2013년12월  17 non-null     int64 
 2   2014년1월   17 non-null     int64 
 3   2014년2월   17 non-null     int64 
 4   2014년3월   17 non-null     int64 
 5   2014년4월   17 non-null     int64 
 6   2014년5월   17 non-null     int64 
 7   2014년6월   17 non-null     int64 
 8   2014년7월   17 non-null     int64 
 9   2014년8월   17 non-null     int64 
 10  2014년9월   17 non-null     int64 
 11  2014년10월  17 non-null     int64 
 12  2014년11월  17 non-null     int64 
 13  2014년12월  17 non-null     int64 
 14  2015년1월   17 non-null     int64 
 15  2015년2월   17 non-null     int64 
 16  2015년3월   17 non-null     int64 
 17  2015년4월   17 non-null     int64 
 18  2015년5월   17 non-null     int64 
 19  2015년6월   17 non-null     int64 
 20  2015년7월   17 non-null     int64 
 21  2015년8월   17 non-n

### '지역별' '기간들의 평당분양가격'을 '지역별' '기간별' '평당분양가'로 변형

#### Melt() <-> Unstack()
- pandas의 melt를 이용해 unstack() 반대방향으로 변형을 할 수 있습니다.
- 각 컬럼의 이름으로 존재하는 데이터들을 행으로 변형하여 조합 또는 어울림의 관계로 배치

In [310]:
# pd.melt를 사용해 펼쳐진 데이터를 지역별 기간별 평당가격으로 세운다.
df_first_melt = df_first.melt(id_vars='지역', var_name='기간', value_name='평당분양가격')
df_first_melt.head(3)

,지역,기간,평당분양가격
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080


In [311]:
df_first_melt.columns = ['지역명', '기간', '평당분양가격']
df_first_melt.head(1)

,지역명,기간,평당분양가격
0,서울,2013년12월,18189


### YYYY년MM월을 YYYY와 MM으로 분리

In [329]:
# {'기간' : 'YYYY년MM월'} -> {'연도': YYYY, '월': MM}
df_first_melt['연도'] = df_first_melt['기간'].apply(lambda x : int(x.split('년')[0]))
df_first_melt['월'] = df_first_melt['기간'].apply(lambda x : int(x.split('년')[-1].replace("월", "")))

In [330]:
# 검증
print(df_first_melt['연도'].unique())
print(df_first_melt['월'].unique())

[2013 2014 2015]
[12  1  2  3  4  5  6  7  8  9 10 11]
